In [1]:
import pandas as pd

support_data_path = "./Datasets/Support Datasets/"
regionalatlas_path = "./Datasets/Regionalatlas/"

### Extraktion README für Mapping Attribute Code ✅

Zuerst werden aus der README_regionalatlas.md. Extrahiert alle Spalten der großen attribute-code-Tabelle aus der README und speichert sie als CSV.

In [2]:
def extract_attribute_table_full(md_path, csv_out):
    """
    Extrahiert alle Spalten der großen attribute-code-Tabelle aus der README
    und speichert sie als CSV.
    """
    rows = []
    in_table = False
    with open(md_path, encoding="utf-8") as f:
        for line in f:
            # Tabelle beginnt mit diesem Header:
            if line.strip().startswith("|attribute-title"):
                in_table = True
                continue  # skip header
            if in_table:
                if line.strip() == "" or not line.strip().startswith("|"):
                    break  # Ende der Tabelle
                parts = [c.strip() for c in line.strip().split("|")[1:-1]]
                # 6 Spalten (könnten manchmal weniger sein, daher check)
                if len(parts) >= 6:
                    row = {
                        "attribute-title": parts[0],
                        "attribute-code": parts[1],
                        "attribute-unit": parts[2],
                        "table-title": parts[3],
                        "table-code": parts[4],
                        "category": parts[5]
                    }
                    # Nur sinnvolle Zeilen nehmen
                    if row["attribute-code"] and row["attribute-title"]:
                        rows.append(row)

    df = pd.DataFrame(rows).drop_duplicates("attribute-code")
    df.to_csv(csv_out, index=False)
    print(f"Mapping geschrieben nach: {csv_out} ({len(df)} Zeilen)")
    return df

In [ ]:
# Anwendung:
df_mapping = extract_attribute_table_full(f"{regionalatlas_path}README_regionalatlas.md", f"{support_data_path}attribute_mapping.csv")

Mapping geschrieben nach: ./Datasets/Support Datasets/attribute_mapping2.csv (202 Zeilen)


### Vorbereitung (muss nicht jedes Mal ausgeführt werden) ✅

Zuerst müssen wir die sogenannten `table-code` der verschiedenen Tabellen des Regionalatlas identifizieren. Es gibt pro Tabelle einen `table-code`. Tabellen sind immer einer Kategorie `category` zugeordnet, es können auch mehrere Tabellen zu einer Kategorie gehören.

Definition der Funktion. Hier wird die Funktion erstellt, die es erlaubt die `table-code` aus der Mapping Tabelle `attribute_mapping.csv` zu ziehen.

In [4]:
def get_table_codes_for_categories(mapping_csv_path: str, categories: list) -> dict:
    df_map = pd.read_csv(mapping_csv_path)
    result = {}
    for cat in categories:
        df_cat = df_map[df_map['category'].str.lower() == cat.lower()]
        unique_tables = df_cat['table-code'].unique()
        if len(unique_tables) == 0:
            print(f"⚠️ Keine table-codes für Kategorie '{cat}' gefunden.")
            result[cat] = []
        else:
            result[cat] = unique_tables.tolist()
    return result


Hier werden die `categories` festgelegt. Wir haben uns auf eine Auswahl relevanter Tabellen beschränkt um nicht irrelevante Daten abzufragen.

In [5]:
categories = ["Bildung", "Bevölkerung", "Verkehr", "Unternehmen", "Erwerbstätigkeit und Arbeitslosigkeit", "Landwirtschaft", "Industrie", "Bauen und Wohnen", "Tourismus", "Verkehr", "Öffentliche Haushalte", "Verdienste und Einkommen", "Bruttoinlandsprodukt und Bruttowertschöpfung", "Nachhaltigkeit", "Soziales" ]

Hier passiert die Abfrage aufgrund der zuvor festgelegten categories.

In [6]:
mapping_file = f"{support_data_path}attribute_mapping.csv"

table_codes = get_table_codes_for_categories(mapping_file, categories)

# Ausgabe
for cat, codes in table_codes.items():
    print(f"Category '{cat}': table-codes: {codes}")

Category 'Bildung': table-codes: ['AI003-1', 'AI003-2', 'AI003-3']
Category 'Bevölkerung': table-codes: ['AI002-1-5', 'AI002-2-5', 'AI002-3', 'AI002-4-5', 'AI002-5']
Category 'Verkehr': table-codes: ['AI013-1', 'AI013-2', 'AI013-3']
Category 'Unternehmen': table-codes: ['AI004-1', 'AI004-2', 'AI004-3']
Category 'Erwerbstätigkeit und Arbeitslosigkeit': table-codes: ['AI007-1', 'AI007-2', 'AI008-1-5', 'AI008-2']
Category 'Landwirtschaft': table-codes: ['AI009']
Category 'Industrie': table-codes: ['AI010-1', 'AI010-2-5']
Category 'Bauen und Wohnen': table-codes: ['AI011-5']
Category 'Tourismus': table-codes: ['AI012-5']
Category 'Öffentliche Haushalte': table-codes: ['AI015']
Category 'Verdienste und Einkommen': table-codes: ['AI016-1', 'AI016-2-5']
Category 'Bruttoinlandsprodukt und Bruttowertschöpfung': table-codes: ['AI017-1', 'AI017-2']
Category 'Nachhaltigkeit': table-codes: ['AI-N-01-2-5', 'AI-N-01-3-5', 'AI-N-01-5', 'AI-N-02', 'AI-N-04', 'AI-N-05', 'AI-N-09', 'AI-N-11']
Category 'S

### Übersicht über Table Title, Table Code und Category ✅

In [7]:
# Alle Codes in eine Liste packen (auch falls mehrere pro Kategorie)
all_selected_codes = [code for code_list in table_codes.values() for code in code_list]


In [8]:
df = pd.read_csv(f"{support_data_path}attribute_mapping.csv")

# DataFrame filtern
df_result = df[df['table-code'].isin(all_selected_codes)][['table-title', 'table-code', 'category']].drop_duplicates()

# Als CSV speichern
df_result.to_csv('selected_tables.csv', index=False)


Ergebnis `df_result` sind alle ausgewählten Tabellen `table-title` inklusive `table-code` und `category`

In [9]:
df_result.head(10)

,table-title,table-code,category
13,Bevölkerungsstand - Geburten - Gestorbene - Wa...,AI002-1-5,Bevölkerung
20,Bevölkerung nach Alter,AI002-2-5,Bevölkerung
25,Wanderungen nach Geschlecht und- Alter,AI002-3,Bevölkerung
27,Bevölkerung - Durchschnittsalter,AI002-4-5,Bevölkerung
29,Einbürgerungen,AI002-5,Bevölkerung
30,Betreute Kinder in Tagespflege/Tageseinrichtungen,AI003-1,Bildung
33,Schulabgänger/-innen,AI003-2,Bildung
35,Betreuungsquote,AI003-3,Bildung
37,Gewerbeanmeldungen,AI004-1,Unternehmen
38,Unternehmensinsolvenzen,AI004-2,Unternehmen


## SQL Abfrage

### Vorbereitung SQL Abfrage ✅

Zuerst werden die Top 30 Städte in die `top30` geladen. Diese werden dann mit den jeweiligen ARS aus `plz_ars.csv` zusammengeführt.

In [10]:
import pandas as pd

top30 = pd.read_csv(f"{support_data_path}top30_staedte.csv", header=None, names=['Stadt'])
plz_ars = pd.read_csv(f"{support_data_path}plz_ars.csv", dtype=str)

ergebnisse = []

# Definiere die Ausnahmestädte, die als Teil von 'Städteregion' oder 'Region' vorkommen können
ausnahmen = ['Aachen', 'Hannover']

for stadt in top30['Stadt']:
    if stadt in ausnahmen:
        # Für Ausnahmen: Der Name kommt irgendwo im Text vor
        regex = fr".*\b{stadt}\b.*"
    else:
        # Normalfall: Beginnt mit Stadtnamen (das deckt auch "Leipzig, Stadt" etc. ab)
        regex = fr"^{stadt}\b"
    matches = plz_ars[plz_ars['Gemeinde/Stadt'].str.contains(regex, case=False, na=False, regex=True)]
    for _, row in matches.iterrows():
        ergebnisse.append({
            'Stadt': stadt,
            'Gemeinde/Stadt': row['Gemeinde/Stadt'],
            'ARS': row['ARS'],
            'Amtlicher Gemeindeschlüssel(AGS)': row['Amtlicher Gemeindeschlüssel(AGS)'],
            'Textkennzeichen': row['Textkennzeichen']
        })

out = pd.DataFrame(ergebnisse)

out.to_csv(f"{support_data_path}top30_staedte_AGS_ARS.csv", index=False)
print(out)

anzahl_staedte = out['Stadt'].nunique()
print(f"Anzahl unterschiedlicher Städte: {anzahl_staedte}")


           Stadt                 Gemeinde/Stadt           ARS  \
0         Berlin                         Berlin            11   
1         Berlin                  Berlin, Stadt         11000   
2         Berlin                  Berlin, Stadt  110000000000   
3        Hamburg                        Hamburg            02   
4        Hamburg  Hamburg, Freie und Hansestadt         02000   
..           ...                            ...           ...   
66  Braunschweig            Braunschweig, Stadt  031010000000   
67          Kiel         Kiel, Landeshauptstadt         01002   
68          Kiel         Kiel, Landeshauptstadt  010020000000   
69      Chemnitz                Chemnitz, Stadt         14511   
70      Chemnitz                Chemnitz, Stadt  145110000000   

   Amtlicher Gemeindeschlüssel(AGS)   Textkennzeichen  
0                               NaN               NaN  
1                               NaN  Kreisfreie Stadt  
2                          11000000        Stadtsta

### Erstellung SQL Query

Nun werden mit der geraden erstellen Funktion alle relevanten SQL Queries gebaut.

#### Funktionen build_sql_query, normalize_table_code, fetch_regionalatlas werden gebaut ✅

In [11]:
import requests
import json
import pandas as pd

def normalize_table_code(table_code: str) -> str:
    return table_code.lower().replace('-', '_')

def build_sql_query(table_code: str, ags: str, jahr: int = 2020, region_typ: int = 3) -> str:
    tbl = normalize_table_code(table_code)
    return (
        f"SELECT * "
        f"FROM verwaltungsgrenzen_gesamt "
        f"LEFT OUTER JOIN {tbl} ON ags = ags2 AND jahr = jahr2 "
        f"WHERE typ = {region_typ} AND jahr = {jahr} AND ags = '{ags}'"
    )

def fetch_regionalatlas(table_code: str, ags: str, jahr: int = 2020, region_typ: int = 3) -> pd.DataFrame:
    tbl = normalize_table_code(table_code)
    where_clause = f"typ = {region_typ} AND jahr = {jahr} AND ags = '{ags}'"
    
    layer_obj = {
        "source": {
            "dataSource": {
                "geometryType": "esriGeometryPolygon",
                "workspaceId": "gdb",
                "query": build_sql_query(table_code, ags, jahr, region_typ),
                "oidFields": "id",
                "spatialReference": {"wkid": 25832},
                "type": "queryTable"
            },
            "type": "dataLayer"
        }
    }

    url = (
        "https://www.gis-idmz.nrw.de/arcgis/rest/services/"
        "stba/regionalatlas/MapServer/dynamicLayer/query"
    )
    params = {
        "layer": json.dumps(layer_obj),
        "where": where_clause,
        "outFields": "*",
        "f": "json"
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if "features" in data:
        features = [feat["attributes"] for feat in data["features"]]
        return pd.DataFrame(features)
    else:
        raise RuntimeError(f"Keine Daten für {table_code}, AGS {ags}: {data}")


##### Beispielaufruf fetch_multiple --> kann weg??

In [12]:
def fetch_multiple(table_code: str, ags_list: list, jahr: int = 2020, region_typ: int = 5):
    dfs = []
    for ags in ags_list:
        try:
            df_tmp = fetch_regionalatlas(table_code, ags, jahr, region_typ)
            df_tmp["AGS"] = ags
            dfs.append(df_tmp)
        except Exception as e:
            print(f"Fehler bei {table_code}, AGS {ags}: {e}")
    return pd.concat(dfs, ignore_index=True)

# Beispiel:
ags_list = ["05513", "05334002", "05334000 "]
df_all = fetch_multiple("AI002-1-5", ags_list, jahr=2020)
print(df_all.head())


         id  typ       ags    jahr            gen   jahr2      ags2  \
0  457086.0  5.0     05513  2020.0  Gelsenkirchen  2020.0     05513   
1  456989.0  5.0  05334002  2020.0         Aachen  2020.0  05334002   

                                    gen2  ai0201  ai0202  ai0208  ai0209  \
0        Gelsenkirchen, kreisfreie Stadt  2469.1   -20.8    21.5    10.9   
1               Aachen, kreisfreie Stadt  1547.3    -3.3    19.4     9.0   

   ai0210  ai0211  ai0212       AGS  
0    14.0   -30.8     9.9     05513  
1    10.0   -10.0     8.7  05334002  


In [14]:
all_dfs = []
for jahr in [2020, 2021, 2022, 2023, 2024, 2025]:
    df = fetch_multiple("AI002-1-5", ags_list, jahr=jahr)
    df["Jahr"] = jahr   # Damit du später weißt, zu welchem Jahr die Zeile gehört
    all_dfs.append(df)

df_gesamt = pd.concat(all_dfs, ignore_index=True)
print(df_gesamt.head())

         id  typ       ags    jahr            gen   jahr2      ags2  \
0  457086.0  5.0     05513  2020.0  Gelsenkirchen  2020.0     05513   
1  456989.0  5.0  05334002  2020.0         Aachen  2020.0  05334002   
2  468122.0  5.0     05513  2021.0  Gelsenkirchen  2021.0     05513   
3  468025.0  5.0  05334002  2021.0         Aachen  2021.0  05334002   
4  479778.0  5.0     05513  2022.0  Gelsenkirchen  2022.0     05513   

                                    gen2  ai0201  ai0202  ai0208  ai0209  \
0        Gelsenkirchen, kreisfreie Stadt  2469.1   -20.8    21.5    10.9   
1               Aachen, kreisfreie Stadt  1547.3    -3.3    19.4     9.0   
2        Gelsenkirchen, kreisfreie Stadt  2478.8    39.3    22.5    11.1   
3               Aachen, kreisfreie Stadt  1548.5     7.7    20.3     8.9   
4        Gelsenkirchen, kreisfreie Stadt  2521.9   109.9    23.3    10.7   

   ai0210  ai0211  ai0212       AGS  Jahr  
0    14.0   -30.8     9.9     05513  2020  
1    10.0   -10.0     8.7  0

In [17]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("regionalatlas_filtered.db")
table = "AI015"  # oder wie du den clean_title gewählt hast
ags_aachen = ["05334000", "05334002"]
query = f"""
SELECT Jahr, ags, AI1501
FROM "{table}"
WHERE ags IN ({','.join(['?']*len(ags_aachen))})
"""
df = pd.read_sql_query(query, conn, params=ags_aachen)
conn.close()
print(df)


DatabaseError: Execution failed on sql '
SELECT Jahr, ags, AI1501
FROM "AI015"
WHERE ags IN (?,?)
': no such table: AI015

In [6]:
df_all = df_gesamt
df_all.head(500)

,id,typ,ags,jahr,gen,jahr2,ags2,gen2,ai0201,ai0202,ai0208,ai0209,ai0210,ai0211,ai0212,AGS,Jahr
0,457086.0,5.0,05513,2020.0,Gelsenkirchen,2020.0,05513,"Gelsenkirchen, kreisfreie Stadt",2469.1,-20.8,21.5,10.9,14.0,-30.8,9.9,05513,2020
1,456989.0,5.0,05334002,2020.0,Aachen,2020.0,05334002,"Aachen, kreisfreie Stadt",1547.3,-3.3,19.4,9.0,10.0,-10.0,8.7,05334002,2020
2,468122.0,5.0,05513,2021.0,Gelsenkirchen,2021.0,05513,"Gelsenkirchen, kreisfreie Stadt",2478.8,39.3,22.5,11.1,14.0,-28.6,64.8,05513,2021
3,468025.0,5.0,05334002,2021.0,Aachen,2021.0,05334002,"Aachen, kreisfreie Stadt",1548.5,7.7,20.3,8.9,9.9,-10.0,21.0,05334002,2021
4,479778.0,5.0,05513,2022.0,Gelsenkirchen,2022.0,05513,"Gelsenkirchen, kreisfreie Stadt",2521.9,109.9,23.3,10.7,13.9,-32.5,132.4,05513,2022
5,479681.0,5.0,05334002,2022.0,Aachen,2022.0,05334002,"Aachen, kreisfreie Stadt",1634.4,122.3,21.8,7.9,10.3,-24.4,142.2,05334002,2022
6,491352.0,5.0,05513,2023.0,Gelsenkirchen,2023.0,05513,"Gelsenkirchen, kreisfreie Stadt",2547.0,99.3,24.3,10.3,13.5,-31.8,133.0,05513,2023
7,491255.0,5.0,05334002,2023.0,Aachen,2023.0,05334002,"Aachen, kreisfreie Stadt",1639.9,33.6,22.6,7.5,9.8,-23.0,52.3,05334002,2023


### Abfrage + Datenbank ✅

In den folgenden Schritten wird eine Datenbank erstellt, in der jede zuvor ausgewählte Tabelle mit den Werten für unsere ausgewählten Gebiete befüllt wird.

In [14]:
# Tabelle für table_code → table-title Mapping
mapping_df = pd.read_csv(f"{support_data_path}selected_tables.csv")
table_codes = mapping_df['table-code'].tolist()


In [16]:
df = pd.read_csv(f"{support_data_path}top30_staedte_AGS_ARS.csv", dtype=str)

# Definiere Sonderfälle
sonderfaelle = ['Aachen', 'Hannover']

def choose_ags(row):
    if row['Stadt'] in sonderfaelle:
        # Nimm Wert aus 'Amtlicher Gemeindeschlüssel(AGS)' (achten auf NaN!)
        return row['Amtlicher Gemeindeschlüssel(AGS)']
    else:
        return row['ARS']

# Wähle den richtigen Schlüssel je Zeile
df['AGS_final'] = df.apply(choose_ags, axis=1)

# Jetzt alle gültigen und eindeutigen AGS extrahieren
ags_list = df['AGS_final'].dropna().unique().tolist()
print(ags_list)


['11', '11000', '110000000000', '02', '02000', '020000000000', '09162', '091620000000', '09184', '05315', '053150000000', '06412', '064120000000', '05111', '051110000000', '08111', '081110000000', '14713', '147130000000', '14729', '05913', '059130000000', '04', '04011', '040110000000', '034530006006', '05113', '051130000000', '14612', '146120000000', '09564', '095640000000', '03241001', '05112', '051120000000', '05911', '059110000000', '05124', '051240000000', '05711', '057110000000', '05314', '053140000000', '08222', '082220000000', '08212', '082120000000', '08215', '05515', '055150000000', '064320015015', '073395001038', '097795725187', '09761', '097610000000', '09772', '06414', '064140000000', '05513', '055130000000', '05116', '051160000000', '05334002', '03101', '031010000000', '01002', '010020000000', '14511', '145110000000']


In [17]:
import pandas as pd
import sqlite3

# Tabelle für table_code → table-title Mapping
mapping_df = pd.read_csv(f"{support_data_path}selected_tables.csv")
table_codes = mapping_df['table-code'].tolist()
code2title = dict(zip(mapping_df['table-code'], mapping_df['table-title']))

conn = sqlite3.connect('regionalatlas_filtered.db')

jahre = [2020, 2021, 2022, 2023, 2024, 2025]

for tbl in table_codes:
    all_dfs = []
    for jahr in jahre:
        for ags in ags_list:
            df_tmp = fetch_regionalatlas(tbl, ags, jahr, region_typ=5)
            df_tmp["ags"] = ags
            df_tmp["jahr"] = jahr
            all_dfs.append(df_tmp)
    # Kombiniere alles zu einem DataFrame
    full_df = pd.concat(all_dfs, ignore_index=True)
    # Sauberer Tablename aus table-title
    raw_title = code2title.get(tbl, tbl)
    clean_title = (
        raw_title.replace(' ', '_')
        .replace('/', '_')
        .replace('(', '')
        .replace(')', '')
    )
    # Schreibe ALLE Jahre gemeinsam weg!
    full_df.to_sql(clean_title, conn, if_exists="replace", index=False)
    print(f"Gespeichert: {clean_title}")

conn.close()


Gespeichert: Bevölkerungsstand_-_Geburten_-_Gestorbene_-_Wanderungen
Gespeichert: Bevölkerung_nach_Alter
Gespeichert: Wanderungen_nach_Geschlecht_und-_Alter
Gespeichert: Bevölkerung_-_Durchschnittsalter
Gespeichert: Einbürgerungen
Gespeichert: Betreute_Kinder_in_Tagespflege_Tageseinrichtungen
Gespeichert: Schulabgänger_-innen
Gespeichert: Betreuungsquote
Gespeichert: Gewerbeanmeldungen


C:\Users\tomsa\AppData\Local\Temp\ipykernel_372\1198911187.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(all_dfs, ignore_index=True)


Gespeichert: Unternehmensinsolvenzen


C:\Users\tomsa\AppData\Local\Temp\ipykernel_372\1198911187.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(all_dfs, ignore_index=True)


Gespeichert: Statistisches_Unternehmensregister:_Abhängig_Beschäftigte_AB_je_1.000_EW


...

Gespeichert: Wirtschaftliche_Leistungsfähigkeit_-_BIP_je_Arbeitsst.


C:\Users\tomsa\AppData\Local\Temp\ipykernel_372\1198911187.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(all_dfs, ignore_index=True)


Gespeichert: Armutsgefährdung


C:\Users\tomsa\AppData\Local\Temp\ipykernel_372\1198911187.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(all_dfs, ignore_index=True)


Gespeichert: Mindestsicherungsleistungen
Gespeichert: Grundsicherung_für_Arbeitssuchende_SGB_II
Gespeichert: Grundsicherung_im_Alter_und_bei_Erwerbsminderung


#### Mapping der Attribute Codes

In [ ]:
import sqlite3
import pandas as pd

mapping = pd.read_csv(f"{support_data_path}attribute_mapping.csv")
code2title = dict(zip(mapping['attribute-code'].str.lower(), mapping['attribute-title']))

conn = sqlite3.connect('regionalatlas_filtered.db')

# Hole alle relevanten Tabellennamen (z. B. aus selected_tables.csv)
table_names = pd.read_csv(f"{support_data_path}selected_tables.csv")['table-title'].apply(
    lambda x: x.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '')
).tolist()

for tablename in table_names:
    try:
        df = pd.read_sql_query(f'SELECT * FROM "{tablename}"', conn)
        new_columns = {}
        for col in df.columns:
            col_lower = col.lower()
            if col_lower in code2title:
                new_columns[col] = code2title[col_lower]
            else:
                new_columns[col] = col
        df_renamed = df.rename(columns=new_columns)
        df_renamed.to_sql(tablename, conn, if_exists="replace", index=False)
        print(f"Spalten in Tabelle '{tablename}' umbenannt!")
    except Exception as e:
        print(f"Tabelle '{tablename}' konnte nicht bearbeitet werden: {e}")

conn.close()


Spalten in Tabelle 'Bevölkerungsstand_-_Geburten_-_Gestorbene_-_Wanderungen' umbenannt!
Spalten in Tabelle 'Bevölkerung_nach_Alter' umbenannt!
Spalten in Tabelle 'Wanderungen_nach_Geschlecht_und-_Alter' umbenannt!
Spalten in Tabelle 'Bevölkerung_-_Durchschnittsalter' umbenannt!
Spalten in Tabelle 'Einbürgerungen' umbenannt!
Spalten in Tabelle 'Betreute_Kinder_in_Tagespflege_Tageseinrichtungen' umbenannt!
Spalten in Tabelle 'Schulabgänger_-innen' umbenannt!
Spalten in Tabelle 'Betreuungsquote' umbenannt!
Spalten in Tabelle 'Gewerbeanmeldungen' umbenannt!
Spalten in Tabelle 'Unternehmensinsolvenzen' umbenannt!
Spalten in Tabelle 'Statistisches_Unternehmensregister:_Abhängig_Beschäftigte_AB_je_1.000_EW' umbenannt!
Spalten in Tabelle 'Erwerbstätige_ET_nach_Wirtschaftsbereichen' umbenannt!
Spalten in Tabelle 'Beschäftigtenquote' umbenannt!
Spalten in Tabelle 'Arbeitslosenquote,_Anteil_Arbeitslose' umbenannt!
Spalten in Tabelle 'Arbeitslosenquote_für_ausgewählte_Personengruppen' umbenannt!
S

#### Aufräumen der DB ✅

Die Spalten aus ´drop_cols´ werden aus der Datenbank entfernt.

In [20]:
drop_cols = ["jahr2", "ags2", "typ"]

In [21]:
import sqlite3
import pandas as pd

# Liste der Tabellen, die du bearbeiten möchtest (wie bisher erzeugt)
table_names = pd.read_csv(f"{support_data_path}selected_tables.csv")['table-title'].apply(
    lambda x: x.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '')
).tolist()

conn = sqlite3.connect('regionalatlas_filtered.db')

for tablename in table_names:
    try:
        df = pd.read_sql_query(f'SELECT * FROM "{tablename}"', conn)
        # Spalten droppen, die im DataFrame vorkommen
        drop_actual = [col for col in drop_cols if col in df.columns]
        if drop_actual:
            df = df.drop(columns=drop_actual)
            df.to_sql(tablename, conn, if_exists="replace", index=False)
            print(f"{tablename}: Spalten {drop_actual} entfernt.")
        else:
            print(f"{tablename}: Keine der Drop-Spalten vorhanden.")
    except Exception as e:
        print(f"Tabelle '{tablename}' konnte nicht bearbeitet werden: {e}")

conn.close()


Bevölkerungsstand_-_Geburten_-_Gestorbene_-_Wanderungen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Bevölkerung_nach_Alter: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Wanderungen_nach_Geschlecht_und-_Alter: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Bevölkerung_-_Durchschnittsalter: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Einbürgerungen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Betreute_Kinder_in_Tagespflege_Tageseinrichtungen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Schulabgänger_-innen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Betreuungsquote: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Gewerbeanmeldungen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Unternehmensinsolvenzen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Statistisches_Unternehmensregister:_Abhängig_Beschäftigte_AB_je_1.000_EW: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Erwerbstätige_ET_nach_Wirtschaftsbereichen: Spalten ['jahr2', 'ags2', 'typ'] entfernt.
Beschäftigtenquote: Spalten ['jahr2', 'ags2', 'typ'] entfern

### Erstellung Datenbank mit kategorisierten Tabellen ✅

Die soeben erstellte db enthält jede einzelne Tabelle als table in der db. Wir wollen diese reduzieren, damit wir nur noch die Kategorien als Tabellen haben und in diesen Tabellen dann alle dieser Kategorie zugehörigen Tabellen vereint sind.

In [22]:
import pandas as pd

selected = pd.read_csv(f"{support_data_path}selected_tables.csv")
# Mapping: Category → [table-title]
category2tables = selected.groupby('category')['table-title'].apply(list).to_dict()


In [23]:
import sqlite3
import pandas as pd
from functools import reduce

# Dein bestehendes Mapping:
# category2tables = {...} wie oben erzeugt

# Neue, gruppierte DB
grouped_db = 'regionalatlas_grouped.db'
conn = sqlite3.connect(grouped_db)

for category, table_names in category2tables.items():
    table_names_clean = [name.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '') for name in table_names]
    dfs = []
    for tablename in table_names_clean:
        try:
            df = pd.read_sql_query(f'SELECT * FROM \"{tablename}\"', sqlite3.connect('regionalatlas_filtered.db'))
            dfs.append(df)
        except Exception as e:
            print(f"Tabelle '{tablename}' konnte nicht geladen werden: {e}")
    if dfs:
        merged = reduce(lambda left, right: pd.merge(left, right, on=['ags', 'jahr'], how='outer', suffixes=('', '_dup')), dfs)
        merged = merged.loc[:, ~merged.columns.str.endswith('_dup')]
        category_table = category.replace(' ', '_').replace('/', '_')
        merged.to_sql(category_table, conn, if_exists='replace', index=False)
        print(f"Neue Tabelle '{category_table}' gespeichert (grouped).")
    else:
        print(f"Keine Tabellen für Category '{category}' gefunden.")

conn.close()


Neue Tabelle 'Bauen_und_Wohnen' gespeichert (grouped).
Neue Tabelle 'Bevölkerung' gespeichert (grouped).
Neue Tabelle 'Bildung' gespeichert (grouped).
Neue Tabelle 'Bruttoinlandsprodukt_und_Bruttowertschöpfung' gespeichert (grouped).
Neue Tabelle 'Erwerbstätigkeit_und_Arbeitslosigkeit' gespeichert (grouped).
Neue Tabelle 'Industrie' gespeichert (grouped).
Neue Tabelle 'Landwirtschaft' gespeichert (grouped).
Neue Tabelle 'Nachhaltigkeit' gespeichert (grouped).
Neue Tabelle 'Soziales' gespeichert (grouped).
Neue Tabelle 'Tourismus' gespeichert (grouped).
Neue Tabelle 'Unternehmen' gespeichert (grouped).
Neue Tabelle 'Verdienste_und_Einkommen' gespeichert (grouped).
Neue Tabelle 'Verkehr' gespeichert (grouped).
Neue Tabelle 'Öffentliche_Haushalte' gespeichert (grouped).
